In [1]:
import numpy as np
import pandas as pd

print("\n\nStarting model training..")
print("--------------------------------------\n")
train_df = pd.read_csv("data.csv")



Starting model training..
--------------------------------------



# Random search code

X = train_df.iloc[:,1:]
y = train_df.iloc[:,0]

from scipy.stats import randint
import scipy
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

scaler = MinMaxScaler(feature_range=(0,1), copy=True)
X = scaler.fit_transform(X)

params = {'max_depth': randint(1, 10) ,
          'learning_rate': scipy.stats.expon(scale=0.5),
          'n_estimators': randint(1, 10),
          'gamma': scipy.stats.expon(scale=1)
           }


import datetime
print("\n\nStarting parameter search..")
print("--------------------------------------\n")
tstart = datetime.datetime.now()
optimizer = RandomizedSearchCV(XGBClassifier(objective = 'multi:softmax'), params, n_iter = 25)
optimizer.fit(X, y)
tstop = datetime.datetime.now()
tdelta = tstop - tstart
print("Finished training.")
print("Training duration in (Days/Hours/Seconds/Milliseconds): {0}".format(tdelta)) 
print(optimizer.score()) #0.91130952380952379
print(optimizer.best_params_) #{'gamma': 0.90390078036156596,
                              #'learning_rate': 0.37483528867120858,
                              #'max_depth': 9,
                              #'n_estimators': 9}

By creating a pipeline we can run one line of code to pre-process our data and train our model.
Later on this pipeline will also enable us to only need 1 line of code to pre-process and make predictions on new data.
Hence the code will be a lot cleaner.

Pipeline.steps can be called to view the all the components and parameters that make up the pipeline.

In [2]:
X = train_df.iloc[:,1:]
y = train_df.iloc[:,0]

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [23]:
# Deeplearning Code
X = train_df.iloc[:,1:]
y = train_df.iloc[:,0]


from sklearn.base import BaseEstimator, TransformerMixin
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier

def create_model():
    model = Sequential()
    #units = (Input+Output)/2
    model.add(Dense(units = (int(round(len(X.columns)+9)/2)), activation = 'relu', input_dim= len(X.columns))) 
    model.add(Dense(10, activation = 'softmax'))
    model.compile(loss=categorical_crossentropy,
                  optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True))
    return model
    
pipeline = Pipeline(steps=[('minmaxscaler', MinMaxScaler(feature_range=(0, 1), copy=True)),
                            ('DeepNet', KerasClassifier(build_fn = create_model, epochs = 150, batch_size = 10))
                           ])

In [24]:
import datetime

tstart = datetime.datetime.now()
pipeline = pipeline.fit(X, y)
tstop = datetime.datetime.now()
tdelta = tstop - tstart
print("Finished training.")
print("Training duration in (/Hours/Minutes/Seconds/Milliseconds): {0}".format(tdelta))
pipeline.steps

Epoch 1/150
2100/2100 [==============================] - 1s 685us/step - loss: 0.7467
Epoch 2/150
2100/2100 [==============================] - 1s 481us/step - loss: 0.3250
Epoch 3/150
2100/2100 [==============================] - 1s 526us/step - loss: 0.2210
Epoch 4/150
2100/2100 [==============================] - 1s 520us/step - loss: 0.1456
Epoch 5/150
2100/2100 [==============================] - 1s 499us/step - loss: 0.1037
Epoch 6/150
2100/2100 [==============================] - 1s 497us/step - loss: 0.0662
Epoch 7/150
2100/2100 [==============================] - 1s 503us/step - loss: 0.0479
Epoch 8/150
2100/2100 [==============================] - 1s 488us/step - loss: 0.0343
Epoch 9/150
2100/2100 [==============================] - 1s 520us/step - loss: 0.0253
Epoch 10/150
2100/2100 [==============================] - 1s 552us/step - loss: 0.0202
Epoch 11/150
2100/2100 [==============================] - 1s 517us/step - loss: 0.0154
Epoch 12/150
2100/2100 [============================

2100/2100 [==============================] - 1s 476us/step - loss: 7.5442e-04
Epoch 95/150
2100/2100 [==============================] - 1s 474us/step - loss: 7.4586e-04
Epoch 96/150
2100/2100 [==============================] - 1s 631us/step - loss: 7.3505e-04
Epoch 97/150
2100/2100 [==============================] - 1s 609us/step - loss: 7.2724e-04
Epoch 98/150
2100/2100 [==============================] - 1s 475us/step - loss: 7.1750e-04
Epoch 99/150
2100/2100 [==============================] - 1s 471us/step - loss: 7.0901e-04
Epoch 100/150
2100/2100 [==============================] - 1s 483us/step - loss: 7.0059e-04
Epoch 101/150
2100/2100 [==============================] - 1s 540us/step - loss: 6.9323e-04
Epoch 102/150
2100/2100 [==============================] - 1s 555us/step - loss: 6.8313e-04
Epoch 103/150
2100/2100 [==============================] - 1s 472us/step - loss: 6.7655e-04
Epoch 104/150
2100/2100 [==============================] - 1s 487us/step - loss: 6.6908e-04
Epoch 1

[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
 ('DeepNet', <keras.wrappers.scikit_learn.KerasClassifier at 0x7fecf6adacc0>)]

Then we train our model

We dont evaluate the model as this is up to the watcher, thus we save it now.

In [30]:
from sklearn.externals import joblib
joblib.dump(pipeline, "pipeline.pkl")
print("Pipeline has been saved as pipeline.pkl")

Pipeline has been saved as pipeline.pkl


Test results
-------------------------------------------------------------

Accuracy: 0.9235714285714286%

Confusion matrix:
[[823   0   5   2   3   6   7   1   5   0]
 [  0 932   2   4   3   4   0   1   2   2]
 [  7   0 739  12  14   3  14   4   4   4]
 [  2  11  17 786   0  32   5  11   9   9]
 [  1   5   5   0 755   0  13   1   6  25]
 [ 10   5   6  15  10 665  11   2  17   6]
 [  7   4   5   0   6  18 785   1   4   1]
 [  2   2  21   3  10   6   0 817   2  21]
 [  5  28  14  16   3  17   5   2 708   4]
 [ 14   4   8  10  27   9   0  16   4 748]]
